<a href="https://colab.research.google.com/github/sayakpaul/keras-convnext-conversion/blob/master/in1k-eval/convert_and_calculate_hashes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ml_collections timm

     |████████████████████████████████| 431 kB 7.5 MB/s 


In [ ]:
!git clone -q https://github.com/sayakpaul/keras-convnext-conversion

In [ ]:
%cd keras-convnext-conversion
!mkdir keras-applications
!mkdir keras-applications/convnext

/content/keras-convnext-conversion


In [ ]:
!python convert_all.py

Converting 224x224 resolution ImageNet-1k models.
  0% 0/5 [00:00<?, ?it/s]Converting convnext_tiny with classification top.
Model: convnext_tiny
Image resolution: 224
Dataset: imagenet-1k
Checkpoint URL: https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth
Instantiating PyTorch model and populating weights...
Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth
100% 109M/109M [00:03<00:00, 32.3MB/s]
Instantiating TensorFlow model...
2022-04-17 06:29:36.175814: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
TensorFlow model instantiated, populating pretrained weights...
Weight population successful, serializing TensorFlow model...
TensorFlow model serialized to: keras-applications/convnext/convnext_tiny.h5...
Converting convnext_tiny without classification top.
Model: convnext_tiny
Image 

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!pwd

/content/keras-convnext-conversion


In [ ]:
!gsutil -m cp -r keras-applications gs://convnext-tf

Copying file://keras-applications/convnext/convnext_tiny_notop.h5 [Content-Type=application/octet-stream]...
Copying file://keras-applications/convnext/convnext_tiny.h5 [Content-Type=application/octet-stream]...
Copying file://keras-applications/convnext/convnext_xlarge_notop.h5 [Content-Type=application/octet-stream]...
Copying file://keras-applications/convnext/convnext_large_notop.h5 [Content-Type=application/octet-stream]...
Copying file://keras-applications/convnext/convnext_base_notop.h5 [Content-Type=application/octet-stream]...
Copying file://keras-applications/convnext/convnext_xlarge.h5 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://clou

In [ ]:
import hashlib
import glob 

all_weights = glob.glob("/content/keras-convnext-conversion/keras-applications/convnext/*.h5")
all_weights = sorted(all_weights)
all_weights

['/content/keras-convnext-conversion/keras-applications/convnext/convnext_base.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_base_notop.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_large.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_large_notop.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_small.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_small_notop.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_tiny.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_tiny_notop.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_xlarge.h5',
 '/content/keras-convnext-conversion/keras-applications/convnext/convnext_xlarge_notop.h5']

In [ ]:
weight_hashes = {}

for weight_file in all_weights:
    architecture = weight_file.split("/")[-1].split(".")[0]
    if "notop" in architecture:
        architecture = architecture.replace("_notop", "")

    # https://www.quickprogrammingtips.com/python/how-to-calculate-sha256-hash-of-a-file-in-python.html
    with open(weight_file,"rb") as f:
        bytes = f.read() # read entire file as bytes
        readable_hash = hashlib.sha256(bytes).hexdigest()

        if architecture not in weight_hashes:
            weight_hashes.update({architecture: [readable_hash]})
        else:
            weight_hashes.get(architecture).append(readable_hash)

In [ ]:
weight_hashes

{'convnext_base': ['d30e0c509f4e1abe2784d33765d4391ce8fbff259b0bd79f4a63684b20db87d2',
  '736f7a96cd933ee568611e29f334737fb9aebaaea021ea7adfe4d2f5cbb4a9aa'],
 'convnext_large': ['8a304c66deb782b0d59837bc13127068901adaaa280cfac604d3341aaf44b2cf',
  'b02b623b3c28586423e6be4aa214e2f5619280b97b4ef6b35ffb686e83235f01'],
 'convnext_small': ['f964ea5cd5618a1e64902a74ca5ccff3797a4fa5dba11a14f2c4d1a562b72f08',
  'fd8f0ac74faa4e364d7cb5b2d32af9ae35b54ce5e80525b5beb7b7571320065a'],
 'convnext_tiny': ['dec324e40ebe943afc7b75b72484646eeb092c04bb079df35911d7080364f9a8',
  '4d4f0e079db2cc0e627b55f7d0d76c367145d14f2c90674415373457cd822346'],
 'convnext_xlarge': ['da65d1294d386c71aebd81bc2520b8d42f7f60eee4414806c60730cd63eb15cb',
  '2bfbf5f0c2b3f004f1c32e9a76661e11a9ac49014ed2a68a49ecd0cd6c88d377']}